# Lab 6 - Coder Agent

A coder agent doesn't necessarily mean that it generates code - but more broadly that it can write code and run it in order to solve a task.

In [1]:
from agents import Agent, Runner, trace, function_tool
import docker
import tempfile
import os
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
client = docker.from_env()
image = "python:3.12-slim"

In [3]:
client.containers.run(image="python:3.12-slim", command=["python", "-c", "print(2+2)"], remove=True)

b'4\n'

In [4]:
@function_tool
def execute_python(code: str) -> str:
    """
    Execute the given Python code inside a Docker container with python:3.12-slim,
    and return whatever is printed to stdout.
    You must print the result of the code to stdout in order to retrieve it.
    This uses the python:3.12-slim image and so it does not have scientific libraries installed;
    write simple python 3.12 code using the standard library only. Do not use numpy or scipy.
    IMPORTANT: You must print the result of the code in order to retrieve it.

    Args:
        code: The Python code to run. Remember to print the result.

    """
    print(f"Executing code: {code}")
    with tempfile.TemporaryDirectory() as tmpdir:
        script_path = os.path.join(tmpdir, "script.py")
        volumes = {tmpdir: {"bind": "/tmp", "mode": "ro"}}
        command = ["python", "/tmp/script.py"]
        with open(script_path, "w") as f:
            f.write(code)
        logs = client.containers.run(image=image, command=command, volumes=volumes, remove=True)
    result = logs.decode("utf-8")
    print(f"Result: {result}")
    return result

In [5]:
print(execute_python.description)

Execute the given Python code inside a Docker container with python:3.12-slim,
and return whatever is printed to stdout.
You must print the result of the code to stdout in order to retrieve it.
This uses the python:3.12-slim image and so it does not have scientific libraries installed;
write simple python 3.12 code using the standard library only. Do not use numpy or scipy.
IMPORTANT: You must print the result of the code in order to retrieve it.


In [6]:
instructions = """
You are a Coder Agent that writes python, uses a tool to execute the code, and returns the results,
in order to solve the user's problem.
"""

In [7]:
number = 5 * 11 * 47 * 307
input = f"What are the prime factors of {number}? Reply only with the answer."

In [8]:
agent = Agent("Coder Agent", model="gpt-4.1-mini", instructions=instructions, tools=[execute_python])

with trace("Coder Agent"):
    result = await Runner.run(agent, input)
    print(result.final_output)

Executing code: def prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return factors

result = prime_factors(793595)
print(result)
Result: [5, 11, 47, 307]

The prime factors of 793595 are 5, 11, 47, and 307.


In [9]:


input = """
Write a python function to calculate the price of an option using the Black-Scholes model.
Then use your tool to execute the code and calculate the price of this option:
GOOGL
Stock Price: 150
Strike Price: 155
Time to maturity: 3 years
Risk free rate: 1%
Volatility: 20%
Dividend Yield: 0%
Respond with the calculated price of the option only.
"""

In [10]:

agent = Agent("Coder Agent", model="gpt-4.1-mini", instructions=instructions, tools=[execute_python])

with trace("Coder Agent"):
    result = await Runner.run(agent, input)
    print(result.final_output)

Executing code: import math
from math import log, sqrt, exp

# Cumulative distribution function for standard normal distribution
from math import erf
def norm_cdf(x):
    return (1.0 + erf(x / sqrt(2.0))) / 2.0

def black_scholes_call(S, K, T, r, sigma, q=0):
    # S: Current stock price
    # K: Strike price
    # T: Time to maturity (in years)
    # r: Risk-free rate
    # sigma: Volatility
    # q: Dividend yield
    d1 = (log(S / K) + (r - q + 0.5 * sigma ** 2) * T) / (sigma * sqrt(T))
    d2 = d1 - sigma * sqrt(T)
    call_price = S * exp(-q * T) * norm_cdf(d1) - K * exp(-r * T) * norm_cdf(d2)
    return call_price

# Given parameters
S = 150
K = 155
T = 3
r = 0.01
sigma = 0.20
q = 0

call_price = black_scholes_call(S, K, T, r, sigma, q)
print(call_price)
Result: 20.44641206078066

The calculated price of the option using the Black-Scholes model is 20.44641206078066.
